## Code

In [1]:
import networkx as nx

BEFORE = 0
OVERLAP = 1

def create_directed_graph(conclusions):
    # Create a directed graph using NetworkX
    G = nx.DiGraph()

    # Iterate through each conclusion to add edges to the graph
    for edge in conclusions:
        # Check the type of edge
        if edge[2] == BEFORE:
            # Add a black edge labeled 'BEFORE'
            G.add_edge(edge[0], edge[1], color='black', label='BEFORE')
        elif edge[2] == OVERLAP:
            # Check if the edge already exists and has a color other than black
            if G.has_edge(edge[0], edge[1]):
                if 'color' not in G[edge[0]][edge[1]] or G[edge[0]][edge[1]]['color'] != 'black':
                    # Add a red edge labeled 'OVERLAP'
                    G.add_edge(edge[0], edge[1], color='red', label='OVERLAP')
            else:
                # Add a new red edge labeled 'OVERLAP'
                G.add_edge(edge[0], edge[1], color='red', label='OVERLAP')

            # Check the reverse direction as well
            if G.has_edge(edge[1], edge[0]):
                if 'color' not in G[edge[1]][edge[0]] or G[edge[1]][edge[0]]['color'] != 'black':
                    # Add a red edge labeled 'OVERLAP' in the reverse direction
                    G.add_edge(edge[1], edge[0], color='red', label='OVERLAP')
            else:
                # Add a new red edge labeled 'OVERLAP' in the reverse direction
                G.add_edge(edge[1], edge[0], color='red', label='OVERLAP')

    return G

def is_consistent(G):
    # Check for consistency in the directed graph
    for cycle in nx.simple_cycles(G):
        # Initialize flags to track the presence of black and red edges in the cycle
        has_black_edge = False
        has_red_edge = False
        prev_vertex = None

        # Iterate through nodes in the cycle
        for node in cycle:
            # Get the first edge of the current node
            current_edge = G[node][next(iter(G[node]))]
            current_vertex = node

            # Check the color of the edge
            if current_edge['color'] == 'black':
                # Update the flag if a black edge is found
                has_black_edge = True
                # Check if the previous edge is also black, indicating inconsistency
                if prev_vertex is not None and G[prev_vertex][current_vertex]['color'] == 'black':
                    return False
            elif current_edge['color'] == 'red':
                # Update the flag if a red edge is found
                has_red_edge = True

            prev_vertex = current_vertex

        # Check if the cycle contains both black and red edges, indicating inconsistency
        if has_black_edge and has_red_edge:
            return False

    # The graph is consistent if no inconsistent cycle is found
    return True


def print_order_with_birth_death(G):
    # Create a temporary graph without red edges for topological sorting
    graph_copy = G.copy()
    red_edges = [(u, v) for u, v, data in G.edges(data=True) if data['color'] == 'red']
    graph_copy.remove_edges_from(red_edges)

    # Use topological sorting to get the order of vertices
    try:
        order = list(nx.topological_sort(graph_copy))

        print("Vertex order (Birth and Death):")

        # Dictionary to store groups of vertices
        vertex_groups = {}

        # Set to store vertices for which the death statement has been printed
        printed_death = set()

        for vertex in order:
            # Check if the vertex has any red edges incident to it in the original graph
            has_red_edges = any(data['color'] == 'red' for _, data in G[vertex].items())

            if has_red_edges:
                # Create or update the group for the current vertex with red edges
                group = vertex_groups.get(vertex, set())
                group.add(vertex)
                neighbors = [neighbor for neighbor, data in G[vertex].items() if data['color'] == 'red']
                group.update(neighbors)
                vertex_groups[vertex] = group
            else:
                # If no red edges, treat it as a separate group
                vertex_groups[vertex] = {vertex}

            # Print "Birth" statement
            print(f"Vertex {vertex} (Birth)")

            # Check if all vertices in the group have been born
            if vertex in vertex_groups:
                # Print "Death" statements for each vertex in the group
                for v in vertex_groups[vertex]:
                    # Check if all vertices in the group have been born before printing death
                    if len(vertex_groups[vertex]) == 1 or all(u in printed_death for u in vertex_groups[vertex]):
                        print(f"Vertex {v} (Death)")
                    else:
                        # Mark the vertex as printed
                        printed_death.add(v)

    except nx.NetworkXUnfeasible:
        print("Graph contains a cycle and cannot be topologically sorted.")

#### Algorithm Explanations

- **create_directed_graph:** Makes a directed graph using the edges given. When the edge is a Before edge, it is treated as a "black" edge and goes from first to second vertex. When the edge is an Overlap edge, two "red" edges are made - an edge from first to second vertex and an edge from second to first vertex. If a black edge already exists, the red edge does not replace it.
- **is_consistent:** Checks for any loops in the graph. The loop must contain at least one black edge, and two red edges between the same two vertices can not be used. For example, if a red edge exists from A to B and from B to A, only 1 of those edges can be used. 
- **print_order_with_birth_death:** Vertices connected by red edge are put into groups together, and vertices without red edges are put into groups on their own. Then births are printed in topological order. Deaths are printed once every vertex in a given group has been born

#### Worst Case Running Times

- **create_directed_graph:** O(E), where E is the number of edges
- **is_consistent:** O(V * (V + E)), where V is the number of vertices and E is the number of edges
- **print_order_with_birth_death:** O(V + E), where V is the number of vertices and E is the number of edges

**Overall:** O(V * (V + E)), as each function is performed once and this is the most time consuming one

## Examples

#### Given Examples

In [4]:
conclusions =  (  ( 0, 1, BEFORE  ),  ( 1, 2, BEFORE  ),  ( 2, 0, OVERLAP  )  ) 

G = create_directed_graph(conclusions)

consistent = is_consistent(G)

print("\033[1mConclusions:\033[0m")
if consistent:
    print("Internally consistent.")
    print_order_with_birth_death(G)
else:
    print("Not internally consistent.")
    
print("")

conclusions1 =  (  ( 0, 1, BEFORE  ),  ( 1, 2, BEFORE  ),  ( 2, 1, OVERLAP  )  ) 

G = create_directed_graph(conclusions1)

consistent = is_consistent(G)

print("\033[1mConclusions1:\033[0m")
if consistent:
    print("Internally consistent.")
    print_order_with_birth_death(G)
else:
    print("Not internally consistent.")

print("")

conclusions2 =  (  ( 0, 1, BEFORE  ),  ( 0, 2, BEFORE  ),  ( 2, 1, OVERLAP  )  ) 

G = create_directed_graph(conclusions2)

consistent = is_consistent(G)

print("\033[1mConclusions2:\033[0m")
if consistent:
    print("Internally consistent.")
    print_order_with_birth_death(G)
else:
    print("Not internally consistent.")
    
print("")

conclusions3 =  (  ( 0, 1, BEFORE  ),  ( 0, 2, BEFORE  ),  ( 2, 1, OVERLAP  ),  ( 3, 0, BEFORE  )  ) 

G = create_directed_graph(conclusions3)

consistent = is_consistent(G)

print("\033[1mConclusions3:\033[0m")
if consistent:
    print("Internally consistent.")
    print_order_with_birth_death(G)
else:
    print("Not internally consistent.")

print("")

conclusions4 =  (  ( 0, 1, BEFORE  ),  ( 0, 2, BEFORE  ),  ( 2, 1, OVERLAP  ),  ( 3, 0, BEFORE  ),  ( 2, 3, BEFORE  )  ) 

G = create_directed_graph(conclusions4)

consistent = is_consistent(G)

print("\033[1mConclusions4:\033[0m")
if consistent:
    print("Internally consistent.")
    print_order_with_birth_death(G)
else:
    print("Not internally consistent.")



Conclusions:
Not internally consistent.

Conclusions1:
Not internally consistent.

Conclusions2:
Internally consistent.
Vertex order (Birth and Death):
Vertex 0 (Birth)
Vertex 0 (Death)
Vertex 1 (Birth)
Vertex 2 (Birth)
Vertex 1 (Death)
Vertex 2 (Death)

Conclusions3:
Internally consistent.
Vertex order (Birth and Death):
Vertex 3 (Birth)
Vertex 3 (Death)
Vertex 0 (Birth)
Vertex 0 (Death)
Vertex 1 (Birth)
Vertex 2 (Birth)
Vertex 1 (Death)
Vertex 2 (Death)

Conclusions4:
Not internally consistent.


#### Example of Failure in Code

5 should die before 2 is born, but it does not. This is due to the fact that the vertex births are printed in topological sort, while the deaths are not printed until a group is complete. Since 2 comes before 0 in topological sort in this example, 0 is born after 2, resulting in 5 being born before 2 is born but dying after 2 is born.

Despite this, it still is calculating whether it is internally or not internally consistent correctly

In [3]:
# Example usage
conclusions = ((8, 1, BEFORE), (0, 1, BEFORE), (5, 2, BEFORE), (2, 1, OVERLAP), (3, 0, BEFORE), (2, 4, BEFORE), (1, 6, BEFORE), (2, 6, BEFORE), (0, 5, OVERLAP))

G = create_directed_graph(conclusions)

consistent = is_consistent(G)

if consistent:
    print("Internally consistent.")
    print_order_with_birth_death(G)
else:
    print("Not internally consistent.")

Internally consistent.
Vertex order (Birth and Death):
Vertex 8 (Birth)
Vertex 8 (Death)
Vertex 5 (Birth)
Vertex 3 (Birth)
Vertex 3 (Death)
Vertex 2 (Birth)
Vertex 0 (Birth)
Vertex 0 (Death)
Vertex 5 (Death)
Vertex 4 (Birth)
Vertex 4 (Death)
Vertex 1 (Birth)
Vertex 1 (Death)
Vertex 2 (Death)
Vertex 6 (Birth)
Vertex 6 (Death)
